In [1]:
import numpy as np
import sys
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import warnings
warnings.filterwarnings('ignore')


# indicate whether you are working in a cloud instance (True if yes, False otherwise)
incloud_access = False

In [2]:
# load some useful cartopy routines
from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# and a new matplotlib routine
import matplotlib.path as mpath

In [3]:
## Import the ecco_v4_py library into Python
## =========================================
##    If ecco_v4_py is not installed in your local Python library,
##    tell Python where to find it.  The example below adds
##    ecco_v4_py to the user's path if it is stored in the folder
##    ECCOv4-py under the user's home directory

from os.path import join,expanduser
user_home_dir = '/Users/aahmed78/ECCOv4-py/'

sys.path.append(join(user_home_dir,'ECCOv4-py'))

import ecco_v4_py as ecco

In [4]:
pwd

'/Users/aahmed78/ECCOv4-py'

In [5]:
ls

LICENSE.txt              ci/                      setup.py
MANIFEST                 ecco_v4_py/              test_ecco_v4_py.sh*
MANIFEST.in              incomplete/              tutorial_plotting.ipynb
README.md                meta_json/
binary_data/             setup.cfg


In [6]:
## Set top-level file directory for the ECCO NetCDF files
## =================================================================

## currently set to ~/Downloads/ECCO_V4r4_PODAAC,
## the default if ecco_podaac_download was used to download dataset granules
ECCO_dir = join(user_home_dir,'Downloads','ECCO_V4r4_PODAAC')

In [7]:
## open ECCO datasets needed for the tutorial

ShortNames_list = ["ECCO_L4_GEOMETRY_LLC0090GRID_V4R4",\
                   "ECCO_L4_SSH_LLC0090GRID_MONTHLY_V4R4",\
                   "ECCO_L4_TEMP_SALINITY_LLC0090GRID_MONTHLY_V4R4"]
if incloud_access == True:
    from ecco_s3_retrieve import ecco_podaac_s3_get_diskaware
    files_dict = ecco_podaac_s3_get_diskaware(ShortNames=ShortNames_list,\
                                              StartDate='2000-01',EndDate='2000-12',\
                                              max_avail_frac=0.5,\
                                              download_root_dir=ECCO_dir)
    ecco_grid = xr.open_dataset(files_dict[ShortNames_list[0]])
    ds_SSH = xr.open_mfdataset(files_dict[ShortNames_list[1]],\
                                     compat='override',data_vars='minimal',coords='minimal')
    ds_temp_sal = xr.open_mfdataset(files_dict[ShortNames_list[2]],\
                                      compat='override',data_vars='minimal',coords='minimal')
else:
    ecco_grid = xr.open_dataset(glob.glob(join(ECCO_dir,ShortNames_list[0],'*.nc'))[0])
    ds_SSH = xr.open_mfdataset(glob.glob(join(ECCO_dir,ShortNames_list[1],'*2000-*.nc')),\
                                     compat='override',data_vars='minimal',coords='minimal')
    ds_temp_sal = xr.open_mfdataset(glob.glob(join(ECCO_dir,ShortNames_list[2],'*2000-*.nc')),\
                                     compat='override',data_vars='minimal',coords='minimal')


## select only *surface* temperature and salinity (SST and SSS)
ds_SST_SSS = ds_temp_sal.isel(k=0)


## Copy ecco_ds from ecco_grid dataset
ecco_ds = ecco_grid.copy()
## Add SSH, SST, and SSS variables to ecco_ds
ecco_ds['SSH'] = ds_SSH['SSH']
ecco_ds['SST'] = ds_SST_SSS['THETA']
ecco_ds['SSS'] = ds_SST_SSS['SALT']

## Load ecco_ds into memory
ecco_ds = ecco_ds.compute()

IndexError: list index out of range